# Exploring the Constant Velocity Physical Warp Drive

This notebook provides an interactive exploration of the constant velocity warp shell metric from paper arXiv:2405.02709v1.

**Paper:** "Constant Velocity Physical Warp Drive Solution"  
**Authors:** Fuchs et al. (2024)  
**Key Result:** First physical warp drive satisfying all energy conditions


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..')

from warpfactory.metrics.warp_shell import get_warp_shell_comoving_metric
from warpfactory.units.constants import c, G

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 4)

## 1. Physical Parameters from the Paper

The paper uses:
- Inner radius: R₁ = 10 m
- Outer radius: R₂ = 20 m  
- Total mass: M = 4.49 × 10²⁷ kg (2.365 Jupiter masses)
- Warp velocity: β = 0.02 (0.02c)

In [ ]:
# Physical parameters
R1 = 10.0  # meters
R2 = 20.0  # meters
M = 4.49e27  # kg
beta_warp = 0.02

# Grid parameters
grid_size = [1, 61, 61, 61]
world_center = [0.0, 30.0, 30.0, 30.0]
grid_scaling = [1.0, 1.0, 1.0, 1.0]

print(f"Shell Configuration:")
print(f"  R1 = {R1} m")
print(f"  R2 = {R2} m")
print(f"  M = {M:.3e} kg ({M/1.898e27:.3f} Jupiter masses)")
print(f"  β_warp = {beta_warp} ({beta_warp*c():.2e} m/s)")

## 2. Create Matter Shell (Section 3 of Paper)

This is a stable spherical shell with:
- Positive ADM mass
- Non-unit lapse function
- Non-flat spatial metric
- Satisfies all energy conditions

In [ ]:
print("Creating matter shell metric...")
shell_metric = get_warp_shell_comoving_metric(
    grid_size=grid_size,
    world_center=world_center,
    m=M,
    R1=R1,
    R2=R2,
    v_warp=0.0,
    do_warp=False,
    grid_scaling=grid_scaling
)
print("✓ Shell metric created")

# Extract parameters
r_vec = shell_metric.params['rVec']
rho_smooth = shell_metric.params['rhoSmooth']
P_smooth = shell_metric.params['PSmooth']
A = shell_metric.params['A']
B = shell_metric.params['B']

## 3. Visualize Shell Properties

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Energy density
rho_energy = rho_smooth * c()**2
axes[0].plot(r_vec, rho_energy, 'b-', linewidth=2)
axes[0].axvline(R1, color='k', linestyle='--', alpha=0.3)
axes[0].axvline(R2, color='k', linestyle='--', alpha=0.3)
axes[0].set_xlabel('r [m]')
axes[0].set_ylabel('Energy Density [J/m³]')
axes[0].set_title('Shell Energy Density')
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim([0, 30])

# Pressure
axes[1].plot(r_vec, P_smooth, 'r-', linewidth=2)
axes[1].axvline(R1, color='k', linestyle='--', alpha=0.3)
axes[1].axvline(R2, color='k', linestyle='--', alpha=0.3)
axes[1].set_xlabel('r [m]')
axes[1].set_ylabel('Pressure [Pa]')
axes[1].set_title('Shell Pressure')
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim([0, 30])

# Metric functions
axes[2].plot(r_vec, -A, 'b-', linewidth=2, label='-g_tt')
axes[2].plot(r_vec, B, 'r-', linewidth=2, label='g_rr')
axes[2].axvline(R1, color='k', linestyle='--', alpha=0.3)
axes[2].axvline(R2, color='k', linestyle='--', alpha=0.3)
axes[2].set_xlabel('r [m]')
axes[2].set_ylabel('Metric Component')
axes[2].set_title('Shell Metric Components')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].set_xlim([0, 30])

plt.tight_layout()
plt.show()

print(f"\nShell Properties:")
print(f"  Peak energy density: {np.max(rho_energy):.3e} J/m³")
print(f"  Peak pressure: {np.max(P_smooth):.3e} Pa")
print(f"  Shell volume: {4/3*np.pi*(R2**3-R1**3):.3e} m³")

## 4. Create Warp Shell (Section 4 of Paper)

Now add a shift vector to create the warp effect while maintaining all energy conditions.

In [ ]:
print("Creating warp shell metric...")
warp_shell_metric = get_warp_shell_comoving_metric(
    grid_size=grid_size,
    world_center=world_center,
    m=M,
    R1=R1,
    R2=R2,
    v_warp=beta_warp,
    do_warp=True,
    grid_scaling=grid_scaling
)
print("✓ Warp shell metric created")

## 5. Compare Shell vs Warp Shell

In [ ]:
# Extract radial slices
t_idx = 0
x_idx = grid_size[1] // 2
z_idx = grid_size[2] // 2

y_indices = np.arange(grid_size[2])
y_coords = (y_indices + 1) * grid_scaling[2] - world_center[2]
r_values = np.abs(y_coords)

# Extract metric components
shell_g00 = shell_metric.tensor[(0,0)][t_idx, x_idx, :, z_idx]
shell_g01 = shell_metric.tensor[(0,1)][t_idx, x_idx, :, z_idx]

warp_g00 = warp_shell_metric.tensor[(0,0)][t_idx, x_idx, :, z_idx]
warp_g01 = warp_shell_metric.tensor[(0,1)][t_idx, x_idx, :, z_idx]

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# g_00 comparison
axes[0].plot(r_values, shell_g00, 'b-', linewidth=2, label='Shell')
axes[0].plot(r_values, warp_g00, 'r--', linewidth=2, label='Warp Shell')
axes[0].axvline(R1, color='k', linestyle='--', alpha=0.3)
axes[0].axvline(R2, color='k', linestyle='--', alpha=0.3)
axes[0].set_xlabel('r [m]')
axes[0].set_ylabel('g_00')
axes[0].set_title('Temporal Metric Component')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# g_01 comparison (shift vector)
axes[1].plot(r_values, shell_g01, 'b-', linewidth=2, label='Shell (no shift)')
axes[1].plot(r_values, warp_g01, 'r-', linewidth=2, label='Warp Shell (with shift)')
axes[1].axvline(R1, color='k', linestyle='--', alpha=0.3)
axes[1].axvline(R2, color='k', linestyle='--', alpha=0.3)
axes[1].set_xlabel('r [m]')
axes[1].set_ylabel('g_01')
axes[1].set_title('Shift Vector Component')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nWarp Effect:")
print(f"  Shift inside shell: {np.min(warp_g01):.4f}")
print(f"  Shift outside shell: {warp_g01[0]:.4f}")
print(f"  Frame dragging velocity: {-np.min(warp_g01)*c():.2e} m/s")

## 6. Key Results

### Achievements:
1. ✓ Matter shell satisfies all energy conditions
2. ✓ Warp shell maintains physicality with shift vector
3. ✓ Constant velocity warp drive is physical
4. ✓ First subluminal physical warp drive with Alcubierre-like transport

### Physical Interpretation:
- The shift vector creates **linear frame dragging** inside the shell
- Passengers inside experience **geodesic motion** (no acceleration)
- The effect is **physical** (not just coordinate transformation)
- Energy density **dominates** momentum flux (maintaining energy conditions)
- Positive **ADM mass** ensures correct asymptotic behavior

## 7. Parameter Exploration

Try changing parameters to see their effects:

In [ ]:
# Try different masses
masses = [M * 0.5, M, M * 2]
colors = ['blue', 'red', 'green']
labels = ['0.5M', 'M', '2M']

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for mass, color, label in zip(masses, colors, labels):
    test_metric = get_warp_shell_comoving_metric(
        grid_size=grid_size,
        world_center=world_center,
        m=mass,
        R1=R1,
        R2=R2,
        v_warp=0.0,
        do_warp=False,
        grid_scaling=grid_scaling
    )
    r_vec_test = test_metric.params['rVec']
    rho_test = test_metric.params['rhoSmooth'] * c()**2
    ax.plot(r_vec_test, rho_test, color=color, linewidth=2, label=label)

ax.axvline(R1, color='k', linestyle='--', alpha=0.3)
ax.axvline(R2, color='k', linestyle='--', alpha=0.3)
ax.set_xlabel('r [m]')
ax.set_ylabel('Energy Density [J/m³]')
ax.set_title('Effect of Mass on Energy Density')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim([0, 30])

plt.tight_layout()
plt.show()

## 8. Conclusion

This notebook demonstrates the groundbreaking result from Fuchs et al. (2024):

**A physical warp drive is theoretically possible!**

By combining:
1. A stable matter shell with positive ADM mass
2. A carefully chosen shift vector distribution
3. Numerical methods (WarpFactory) for complex spacetime analysis

The paper shows that constant velocity warp drives can satisfy all energy conditions while providing genuine geodesic transport.

### Next Steps:
- Solve the acceleration problem
- Optimize mass requirements
- Explore alternative geometries
- Investigate engineering feasibility

### References:
1. Fuchs et al., "Constant Velocity Physical Warp Drive Solution," arXiv:2405.02709v1, 2024
2. Helmerich et al., "Analyzing warp drive spacetimes with Warp Factory," CQG 41:095009, 2024